In [0]:
#the "!" is used for executing commands in terminal
!pip install -q keras
!pip install -U -q PyDrive
import pandas as pd
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, CuDNNGRU, Dropout, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import keras.optimizers
import os
from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from html import unescape

In [0]:
#change this value depending on what you need
use_existing_weights = False
filepath = 'weights_best.hdf5'
model_name = 'rnn_model_twitter.h5'

def authenticate_drive():
  # Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  
  return drive

if use_existing_weights:
  drive = authenticate_drive()
    
  #download weights from google drive
  weight_id = '1cM_aXUHcVylDRu9n57L-6Q0L8XmdCvJU'
  weight_drive_file = drive.CreateFile({'id': weight_id})
  weight_drive_file.GetContentFile(filepath)

In [0]:
!mkdir data
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip && unzip glove.twitter.27B.zip -d data/
!wget http://thinknook.com/wp-content/uploads/2012/09/Sentiment-Analysis-Dataset.zip && unzip Sentiment-Analysis-Dataset.zip -d data/

In [0]:
#load data to the working environment
data_path = os.path.join(os.path.expanduser('~'), 'data', 'Sentiment Analysis Dataset.csv')
dataset = pd.read_csv(data_path, error_bad_lines=False, encoding='utf-8')
dataset.dropna(axis=0, inplace=True)
dataset = dataset.rename(index=str, columns={"SentimentText": "text", "Sentiment": "sentiment"})

embedding_path = os.path.join(os.path.expanduser('~'), 'data', 'glove.twitter.27B.200d.txt')
dim_size = 200

#shuffle data
dataset = dataset.sample(frac=1).reset_index(drop=True)

#take sample of it
dataset = dataset.sample(frac=0.01, random_state=42)

In [0]:
#preprocess dataset
eyes = r"[8:=;]"
nose = r"['`\-]?"

#decode html entities
dataset.text = dataset.text.apply(lambda x: unescape(x))

#fix this
# ã?Ÿã?„ã?“ã‚Œã‚“ã?—ã‚…ã?† at index 1502

dataset['text'] = dataset['text']\
.str.replace(r'https?:\/\/\S+\b|www\.(\w+\.)+\S*', '<url>')\
  .str.replace(r'@\w+', '<user>')\
  .str.replace(r'{}{}[)dD]+|[)dD]+{}{}'.format(eyes, nose, nose, eyes), '<smile>')\
  .str.replace(r'{}{}p+'.format(eyes, nose), '<lolface>')\
  .str.replace(r'{}{}\(+|\)+{}{}'.format(eyes, nose, nose, eyes), '<sadface>')\
  .str.replace(r'{}{}[\/|l*]'.format(eyes, nose), '<neutralface>')\
  .str.replace(r'/',' / ')\
  .str.replace(r'<3','<heart>')\
  .str.replace(r'[-+]?[.\d]*[\d]+[:,.\d]*', '<number>')\
  .str.replace(r'#\S+', '<hashtag>')\
  .str.replace(r'([!?.]){2,}', r'\1 <repeat>')\
  .str.replace(r'\b(\S*?)(.)\2{2,}\b', r'\1\2 <elong>')
  
#source: https://gist.github.com/tokestermw/cb87a97113da12acb388

In [0]:
#prep the dataset
#filter these things from the text
token = text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\]^_`{|}~\t\n')
max_len = dataset['text'].str.len().max()

#learn the vocabulary from all the text
token.fit_on_texts(list(dataset['text']))
vocab_size = len(token.word_index) + 1

#this might produce some error, test these 2 lines
x_train, x_val, y_train, y_val = train_test_split(dataset['text'], dataset['sentiment'], test_size=0.03, shuffle=False)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5)

#encode
x_train_enc = token.texts_to_sequences(x_train)
x_test_enc = token.texts_to_sequences(x_test)

#add zero padding
x_train_enc_pad = sequence.pad_sequences(x_train_enc, maxlen=max_len)
x_test_enc_pad = sequence.pad_sequences(x_test_enc, maxlen=max_len)

In [0]:
#create embedding dictionary
embeddings_index = dict()
f = open(embedding_path)
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

#map the vocabulary to it's word embedding
embedding_matrix = np.zeros((vocab_size, dim_size))
for word, i in token.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector


In [15]:
#checking if data works correctly
index = random.randint(1, x_train.shape[0])

print('Preprocessed sentence')
print(dataset.iloc[index, 3], end='\n\n')
print('Sentiment')
print(y_train[index], end='\n\n')
print('Encoded text')
print(x_train_enc[index], end='\n\n')


res = dict((v,k) for k,v in token.word_index.items())
for num in x_train_enc[index]:
  print(res.get(num), end=' ')

print()
print('Zero padding')
print(x_train_enc_pad[index])

Preprocessed sentence
Wanted Kate to win the Aprentice 

Sentiment
0

Encoded text
[355, 1226, 3, 449, 4, 14404]

wanted kate to win the aprentice 
Zero padding
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0

In [18]:
#deep learning
finished_training = False
model = Sequential()
model.add(Embedding(vocab_size, dim_size, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(CuDNNGRU(32, return_sequences=True))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(CuDNNGRU(16, return_sequences=True))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(GlobalMaxPooling1D())

model.add(Dense(1, activation='sigmoid'))
if use_existing_weights:
  model.load_weights(filepath)

#hyperparameter is a mess, tune it
#lr, batch_size, epoch, dropout, maybe some decay, hidden units is maybe too small
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

earlystop = EarlyStopping(monitor='loss', patience=10, verbose=1, mode='auto')
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='auto')
callback_lists = [earlystop, checkpoint]

model.fit(x_train_enc_pad, y_train, epochs=300, batch_size=100, callbacks=callback_lists)
scores = model.evaluate(x_test_enc_pad, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

model.save(model_name)
finished_training = True

Epoch 1/300
12628/12628 [==============================] - 16s 1ms/step - loss: 1.0142 - acc: 0.5672

Epoch 00001: loss improved from inf to 1.01416, saving model to weights_best.hdf5
Epoch 2/300
12628/12628 [==============================] - 15s 1ms/step - loss: 0.6336 - acc: 0.6433

Epoch 00002: loss improved from 1.01416 to 0.63355, saving model to weights_best.hdf5
Epoch 3/300
11000/12628 [=========================>....] - ETA: 1s - loss: 0.5862 - acc: 0.6857

12628/12628 [==============================] - 15s 1ms/step - loss: 0.5849 - acc: 0.6869

Epoch 00003: loss improved from 0.63355 to 0.58486, saving model to weights_best.hdf5
Epoch 4/300
12628/12628 [==============================] - 15s 1ms/step - loss: 0.5524 - acc: 0.7188

Epoch 00004: loss improved from 0.58486 to 0.55238, saving model to weights_best.hdf5
Epoch 5/300
12628/12628 [==============================] - 15s 1ms/step - loss: 0.5282 - acc: 0.7357

Epoch 00005: loss improved from 0.55238 to 0.52825, saving model to weights_best.hdf5
Epoch 6/300
  700/12628 [>.............................] - ETA: 14s - loss: 0.5272 - acc: 0.7486

12628/12628 [==============================] - 15s 1ms/step - loss: 0.5150 - acc: 0.7459

Epoch 00006: loss improved from 0.52825 to 0.51497, saving model to weights_best.hdf5
Epoch 7/300
12628/12628 [==============================] - 15s 1ms/step - loss: 0.5063 - acc: 0.7536

Epoch 00007: loss improved from 0.51497 to 0.50629, saving model to weights_best.hdf5
Epoch 8/300
11900/12628 [===========================>..] - ETA: 0s - loss: 0.4945 - acc: 0.7639

12628/12628 [==============================] - 15s 1ms/step - loss: 0.4940 - acc: 0.7650

Epoch 00008: loss improved from 0.50629 to 0.49401, saving model to weights_best.hdf5
Epoch 9/300
12628/12628 [==============================] - 15s 1ms/step - loss: 0.4876 - acc: 0.7665

Epoch 00009: loss improved from 0.49401 to 0.48757, saving model to weights_best.hdf5
Epoch 10/300
12628/12628 [==============================] - 15s 1ms/step - loss: 0.4805 - acc: 0.7669

Epoch 00010: loss improved from 0.48757 to 0.48047, saving model to weights_best.hdf5
Epoch 11/300
  700/12628 [>.............................] - ETA: 13s - loss: 0.4728 - acc: 0.7829

12628/12628 [==============================] - 15s 1ms/step - loss: 0.4660 - acc: 0.7822

Epoch 00011: loss improved from 0.48047 to 0.46600, saving model to weights_best.hdf5
Epoch 12/300
12628/12628 [==============================] - 15s 1ms/step - loss: 0.4666 - acc: 0.7787

Epoch 00012: loss did not improve
Epoch 13/300
12628/12628 [==============================] - 14s 1ms/step - loss: 0.4577 - acc: 0.7843

Epoch 00013: loss improved from 0.46600 to 0.45773, saving model to weights_best.hdf5
Epoch 14/300
  300/12628 [..............................] - ETA: 13s - loss: 0.4403 - acc: 0.7833

...

12628/12628 [==============================] - 13s 1ms/step - loss: 0.0483 - acc: 0.9806

Epoch 00136: loss did not improve
Epoch 137/300
12628/12628 [==============================] - 13s 1ms/step - loss: 0.0502 - acc: 0.9815

Epoch 00137: loss did not improve
Epoch 138/300
12628/12628 [==============================] - 13s 1ms/step - loss: 0.0447 - acc: 0.9835

Epoch 00138: loss improved from 0.04826 to 0.04470, saving model to weights_best.hdf5
Epoch 139/300
 2600/12628 [=====>........................] - ETA: 10s - loss: 0.0396 - acc: 0.9854

12628/12628 [==============================] - 13s 1ms/step - loss: 0.0421 - acc: 0.9839

Epoch 00139: loss improved from 0.04470 to 0.04214, saving model to weights_best.hdf5
Epoch 140/300
12628/12628 [==============================] - 13s 1ms/step - loss: 0.0470 - acc: 0.9829

Epoch 00140: loss did not improve
Epoch 141/300
12628/12628 [==============================] - 13s 1ms/step - loss: 0.0542 - acc: 0.9797

Epoch 00141: loss did not improve
Epoch 142/300
  200/12628 [..............................] - ETA: 13s - loss: 0.0357 - acc: 0.9800

12628/12628 [==============================] - 13s 1ms/step - loss: 0.0553 - acc: 0.9800

Epoch 00142: loss did not improve
Epoch 143/300
12628/12628 [==============================] - 13s 1ms/step - loss: 0.0555 - acc: 0.9800

Epoch 00143: loss did not improve
Epoch 144/300
12628/12628 [==============================] - 13s 1ms/step - loss: 0.0559 - acc: 0.9798

Epoch 00144: loss did not improve
Epoch 145/300
 2600/12628 [=====>........................] - ETA: 10s - loss: 0.0531 - acc: 0.9788

12628/12628 [==============================] - 13s 1ms/step - loss: 0.0496 - acc: 0.9814

Epoch 00145: loss did not improve
Epoch 146/300
12628/12628 [==============================] - 13s 1ms/step - loss: 0.0497 - acc: 0.9815

Epoch 00146: loss did not improve
Epoch 147/300
12628/12628 [==============================] - 13s 1ms/step - loss: 0.0441 - acc: 0.9836

Epoch 00147: loss did not improve
Epoch 148/300
 2400/12628 [====>.........................] - ETA: 10s - loss: 0.0371 - acc: 0.9892

12628/12628 [==============================] - 13s 1ms/step - loss: 0.0539 - acc: 0.9800

Epoch 00148: loss did not improve
Epoch 149/300
12628/12628 [==============================] - 13s 1ms/step - loss: 0.0457 - acc: 0.9834

Epoch 00149: loss did not improve
Epoch 00149: early stopping
3158/3158 [==============================] - 3s 1ms/step

acc: 72.77%


In [0]:
drive = authenticate_drive()

uploaded = drive.CreateFile({'title': filepath})
uploaded.SetContentFile(filepath)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))
print('Done uploading weights')

if finished_training:
  uploaded_model = drive.CreateFile({'title': model_name})
  uploaded_model.SetContentFile(model_name)
  uploaded_model.Upload()
  print('Uploaded file with ID {}'.format(uploaded_model.get('id')))
  print('Done uploading model')

In [0]:
#using the model

done = False
while done == False:
  sentence = input('Input sentence: ')
  
  sentence_encoding = token.texts_to_sequences([sentence])
  padded_sentence = sequence.pad_sequences(sentence_encoding, maxlen=max_len)
  
  prediction = model.predict(np.array(padded_sentence))
  if prediction[0] == 1:
    print(prediction[0])
    print('Positive')
  elif prediction[0] == 0:
    print(prediction[0])
    print('Negative')
  
  finish = input('Do you still want to input another text? [y/n]')
  if finish == 'N' or finish == 'n':
    done = True